In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("cems.db")
cur = con.cursor()

In [20]:
cur.execute("select * from test where substr(test.op_date, -4) is cast(2001 as varchar) LIMIT 5" )
print(cur.fetchall())

[(u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 1, 1.0, 706.0, u'', 11774.7, u'Measured', 1.64, u'Calculated', 0.359, u'Measured', 2577.369, u'Calculated', 736.6, u'Measured', 0.103, u'Calculated', 7179.3), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 2, 1.0, 704.0, u'', 11156.9, u'Measured', 1.592, u'Calculated', 0.372, u'Measured', 2606.306, u'Calculated', 718.8, u'Measured', 0.103, u'Calculated', 7006.2), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 3, 1.0, 702.0, u'', 10990.0, u'Measured', 1.574, u'Calculated', 0.367, u'Measured', 2561.697, u'Calculated', 716.2, u'Measured', 0.103, u'Calculated', 6980.1), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 4, 1.0, 702.0, u'', 11027.6, u'Measured', 1.58, u'Calculated', 0.377, u'Measured', 2631.762, u'Calculated', 716.2, u'Measured', 0.103, u'Calculated', 6980.8), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 5, 1.0, 699.0, u'', 11258.5, u'Measured', 1.601, u'Calculated', 0.381, u'Measured', 2678.697, u'Calcu

In [7]:
cur.execute("select * from test where strftime('%Y', test.op_date) = 2001 LIMIT 5" )

In [12]:
print(cur.fetchall())

[(u'01-01-2008',), (u'01-01-2008',), (u'01-01-2008',), (u'01-01-2008',), (u'01-01-2008',)]


In [22]:
cur.execute("select * from test, labels where substr(test.op_date, -4) is cast(labels.year as varchar) \
AND labels.orispl_code is test.orispl_code AND test.unitid is labels.unitid")

[(u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 1, 1.0, 706.0, u'', 11774.7, u'Measured', 1.64, u'Calculated', 0.359, u'Measured', 2577.369, u'Calculated', 736.6, u'Measured', 0.103, u'Calculated', 7179.3, u'WA', u'Centralia', 3845, u'BW21', 2001, u'Lewis', u'WA041', 46.7559, -122.8578, u'', u'TransAlta', u'ARP : Woolley, Richard C (350) (Ended Sep 20, 2001)<br>Chambers, Linda  (2423) (Started Sep 21, 2001)', u'ARP : Enns, Lonnie S (1870)', u'Tangentially-fired', u'Coal', u'', u'')]


In [2]:
%load_ext sql

In [28]:
%sql sqlite:///cems.db
%%sql
create table data as
select * from test, labels where substr(test.op_date, -4) is cast(labels.year as varchar) 
AND labels.orispl_code is test.orispl_code AND test.unitid is labels.unitid

Done.


[]

In [29]:
% sql select * from data limit 5

Done.


state,facility_name,orispl_code,unitid,op_date,op_hour,op_time,gload,sload,SO2_MASS,SO2_MASS_MEASURE_FLG,SO2_RATE,SO2_RATE_MEASURE_FLG,NOX_RATE,NOX_RATE_MEASURE_FLG,NOX_MASS,NOX_MASS_MEASURE_FLG,CO2_MASS,CO2_MASS_MEASURE_FLG,CO2_RATE,CO2_RATE_MEASURE_FLG,HEAT_INPUT,state:1,facility_name:1,orispl_code:1,unitid:1,year,county,county_code,latitude,longitude,owner,operator,representative_primary,representative_second,unit_type,fuel_type,fule_type_secondary,max_hourly_rate
WA,Centralia,3845,BW21,01-01-2001,1,1.0,706.0,,11774.7,Measured,1.64,Calculated,0.359,Measured,2577.369,Calculated,736.6,Measured,0.103,Calculated,7179.3,WA,Centralia,3845,BW21,2001,Lewis,WA041,46.7559,-122.8578,,TransAlta,"ARP : Woolley, Richard C (350) (Ended Sep 20, 2001)<br>Chambers, Linda (2423) (Started Sep 21, 2001)","ARP : Enns, Lonnie S (1870)",Tangentially-fired,Coal,,
WA,Centralia,3845,BW21,01-01-2001,2,1.0,704.0,,11156.9,Measured,1.592,Calculated,0.372,Measured,2606.306,Calculated,718.8,Measured,0.103,Calculated,7006.2,WA,Centralia,3845,BW21,2001,Lewis,WA041,46.7559,-122.8578,,TransAlta,"ARP : Woolley, Richard C (350) (Ended Sep 20, 2001)<br>Chambers, Linda (2423) (Started Sep 21, 2001)","ARP : Enns, Lonnie S (1870)",Tangentially-fired,Coal,,
WA,Centralia,3845,BW21,01-01-2001,3,1.0,702.0,,10990.0,Measured,1.574,Calculated,0.367,Measured,2561.697,Calculated,716.2,Measured,0.103,Calculated,6980.1,WA,Centralia,3845,BW21,2001,Lewis,WA041,46.7559,-122.8578,,TransAlta,"ARP : Woolley, Richard C (350) (Ended Sep 20, 2001)<br>Chambers, Linda (2423) (Started Sep 21, 2001)","ARP : Enns, Lonnie S (1870)",Tangentially-fired,Coal,,
WA,Centralia,3845,BW21,01-01-2001,4,1.0,702.0,,11027.6,Measured,1.58,Calculated,0.377,Measured,2631.762,Calculated,716.2,Measured,0.103,Calculated,6980.8,WA,Centralia,3845,BW21,2001,Lewis,WA041,46.7559,-122.8578,,TransAlta,"ARP : Woolley, Richard C (350) (Ended Sep 20, 2001)<br>Chambers, Linda (2423) (Started Sep 21, 2001)","ARP : Enns, Lonnie S (1870)",Tangentially-fired,Coal,,
WA,Centralia,3845,BW21,01-01-2001,5,1.0,699.0,,11258.5,Measured,1.601,Calculated,0.381,Measured,2678.697,Calculated,721.4,Measured,0.103,Calculated,7030.7,WA,Centralia,3845,BW21,2001,Lewis,WA041,46.7559,-122.8578,,TransAlta,"ARP : Woolley, Richard C (350) (Ended Sep 20, 2001)<br>Chambers, Linda (2423) (Started Sep 21, 2001)","ARP : Enns, Lonnie S (1870)",Tangentially-fired,Coal,,


In [30]:
%sql select count(*) from data

Done.


count(*)
120815737


In [3]:
%sql sqlite:///smaller.db


u'Connected: None@smaller.db'

In [4]:
%sql drop table data

(sqlite3.OperationalError) no such table: data [SQL: u'drop table data']


In [ ]:
%sql sqlite:///cems.db
%sql select * from data where state is UPPER('wa') limit 5

In [6]:
%sql ATTACH DATABASE 'smaller.db' AS other;

Done.


[]

In [ ]:
%%sql
create table other.data as
select state, facility_name as name, orispl_code, unitid, op_date, op_hour, op_time, gload, SO2_MASS, NOX_MASS, CO2_MASS,
HEAT_INPUT as heat_input, county, latitude, longitude, fuel_type from data

Done.


[]

In [6]:
for state in ['wa', 'or', 'ca', 'id', 'nv', 'ut', 'az', 'nm', 'co', 'wy', 'mt', 'tx']:
    con = sqlite3.connect(state + ".db")
    con = sqlite3.connect("cems.db")
    cur = con.cursor()
    cur.execute("attach database '" + state + ".db' AS " + state + ";")
    cur.execute("create table " + state + ".data as \
    select state, facility_name as name, orispl_code, unitid, op_date, op_hour, op_time, gload, SO2_MASS, NOX_MASS, CO2_MASS, \
HEAT_INPUT as heat_input, county, latitude, longitude, fuel_type from data where data.state = " + UPPER('state') + ";")

OperationalError: no such column: wa